# Are epistemic and moral virtues semantically distinct?

I investigate whether moral and epistemic concepts are semantically distinct in ordinary language. Some philosophers argue that these concepts overlap considerably, while others maintain that they are fundamentally different. To empirically explore this issue, we employ word embeddings of moral and epistemic virtue terms to examine the contextual usage of these terms. The methodology involves calculating the cosine similarity between word embeddings, conducting clustering analyses, and performing statistical tests to assess the distinctiveness of these concepts.

We first demonstrate that the methodology works by showing that words representing big-5 personality traits form distinct clusters. Then we apply the technique to moral and epistemic virtue words. We find two clearly distinct clusters. This shows that moral and epistemic virtues are semantically distinct.

In [18]:
import openai
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nbformat
from dotenv import load_dotenv
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import push_notebook
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
import ast

In [2]:
# Functions
def get_embedding(word):
    """
    Given a word, return its embedding using OpenAI API.
    """
    response = openai.embeddings.create(
        model="text-embedding-3-large",
        input=word
    )
    return response.data[0].embedding

def apply_get_embedding(df, column_name):
    """
    Apply get_embedding function to a column in a pandas dataframe row-wise.
    """
    df['embedding'] = df[column_name].apply(get_embedding)
    return df

def extract_pca_loadings(df, embedding_column, n_components=2):
    """
    Apply PCA to a column of word embeddings and add the PCA loadings to the dataframe.
    """
    pca = PCA(n_components=n_components)
    embeddings = np.array(df[embedding_column].tolist())
    pca_loadings = pca.fit_transform(embeddings)
    for i in range(n_components):
        df[f'PCA_{i+1}'] = pca_loadings[:, i]
    return df, pca

def plot_pca(df, word_column, category_column, pca_columns):
    """
    Create a 2D graph of the PCA loadings with words and categories.
    """
    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        x=df[pca_columns[0]], 
        y=df[pca_columns[1]], 
        hue=df[category_column], 
        palette='tab10', 
        s=100, 
        alpha=0.7,
        legend='full'
    )
        
    # Annotate each point with the word
    #for i in range(df.shape[0]):
    #    plt.text(df[pca_columns[0]].iloc[i], df[pca_columns[1]].iloc[i], df[word_column].iloc[i], 
    #             fontsize=9, alpha=0.9)
    
    plt.title('PCA of Word Embeddings')
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    plt.legend(title=category_column)
    plt.grid(True)
    plt.show()


def plot_pca_interactive(df, word_column, category_column, pca_columns):
    """
    Create an interactive 2D graph of the PCA loadings with words and categories using Bokeh.
    """
    categories = df[category_column].unique().tolist()
    num_categories = len(categories)
    
    if num_categories == 2:
        palette = ["#1f77b4", "#ff7f0e"]  # Default colors for two categories
    elif num_categories <= 10:
        palette = Category10[num_categories]
    else:
        palette = Category20[num_categories]
    
    source = ColumnDataSource(data=dict(
        x=df[pca_columns[0]],
        y=df[pca_columns[1]],
        word=df[word_column],
        category=df[category_column]
    ))

    hover = HoverTool(tooltips=[
        ("Word", "@word"),
        ("Category", "@category"),
    ])

    p = figure(title="PCA of Word Embeddings", tools=[hover, "pan,wheel_zoom,box_zoom,reset"])
    p.scatter('x', 'y', source=source, legend_field='category', fill_alpha=0.6, size=10,
              color=factor_cmap('category', palette=palette, factors=categories))
    
    p.xaxis.axis_label = "PCA 1"
    p.yaxis.axis_label = "PCA 2"
    p.legend.title = category_column
    p.legend.location = "top_left"

    output_notebook()
    show(p, notebook_handle=True)


## Big Five

Let's begin with showing the methodology using words associated with the big five personality traits. I used GPT to create a database of 30 adjectives associated with each of the Big-five personality traits. Below is a sample of the dataset:

In [19]:
# Load the Big Six Adjectives dataset
df_big_five = pd.read_csv('./big_five_adjectives.csv')
df_big_five.groupby('Category').head(2)

,Adjective,Category
0,creative,Openness to Experience
1,imaginative,Openness to Experience
30,organized,Conscientiousness
31,efficient,Conscientiousness
60,outgoing,Extraversion
61,sociable,Extraversion
90,friendly,Agreeableness
91,kind,Agreeableness
120,anxious,Neuroticism
121,fearful,Neuroticism


I use OpenAI to extract word embeddings for each of the words. Each word embedding is an array with 3072 dimensions. In the next step, I reduce dimensionality to 2 dimensions using principal component analysis. Below is an interactive plot of the embeddings. 

In [8]:
# Apply the embedding extraction
#df = apply_get_embedding(df, 'Adjective')
df = pd.read_csv("./big_five_cum_embeddings.csv")
df['embedding'] = df['embedding'].apply(ast.literal_eval)


In [23]:
# Extract PCA loadings
df, pca = extract_pca_loadings(df, 'embedding')

In [24]:
# Plot the PCA loadings
plot_pca_interactive(df, 'Adjective', 'Category', ['PCA_1', 'PCA_2'])

Loading BokehJS ...

The clusters are clearly visible, showing that the words associated with each of the Big-5 personality traits are indeed semantically distinct. This result can be made more precise by computing cosine similarities within and between word in different categories.  

## 

## Epistemic vs. Moral virtues 

Now we use the same methodology to investigate whether moral and epistemic virtues are semantically distinct. First, I asked GPT to constract a dataset with 50 adjectives associated with epistemic virtue, and 50 adjectives associated with moral virtue. Below is a sample: 

In [26]:
# Load the Big Six Adjectives dataset
df_moral_epistemic = pd.read_csv('./moral_epistemic_adjectives.csv')
df_moral_epistemic.groupby('Category').head(10)

,Adjective,Category
0,honest,Moral
1,kind,Moral
2,just,Moral
3,brave,Moral
4,compassionate,Moral
5,humble,Moral
6,loyal,Moral
7,generous,Moral
8,fair,Moral
9,benevolent,Moral


Following the same process of extracting word embeddings, reducing dimensionality via PCA, and plotting the results, we arrive at the following graph:

In [27]:
# Apply the embedding extraction
#df2 = apply_get_embedding(df2, 'Adjective')
df2 = pd.read_csv('./moral_epistemic_cum_embeddings.csv')
df2['embedding'] = df2['embedding'].apply(ast.literal_eval)


In [28]:
# Extract PCA loadings
df2, pca2 = extract_pca_loadings(df2, 'embedding')

In [29]:
# Plot the PCA loadings
plot_pca_interactive(df2, 'Adjective', 'Category', ['PCA_1', 'PCA_2'])


Loading BokehJS ...

The graph clearly shows that there are semantic differences between epistemic and moral virtues, with very little overlap. Epistemic and moral virtues are indeed semantically distinct.

## Next steps

- Better corpus of epistemic and moral terms
- Add quantitative metrics
- Compare conceptual, semantic, and psychological distinctions 
- Relate to existing literature on epistemic and moral virtues.